# 03 — Maîtriser le paramètre `inplace` en pandas

Le paramètre `inplace=True` permet de modifier directement un DataFrame
sans créer une nouvelle copie.

Cependant, son comportement peut prêter à confusion :
- certaines méthodes retournent un nouvel objet,
- d'autres modifient l'objet existant,
- et l'utilisation de `inplace` peut conduire à des erreurs silencieuses.

Ce notebook illustre ces comportements et propose une recommandation pratique.


In [22]:
import pandas as pd
from pathlib import Path

fp = Path("../data/bitcoins.csv")
df = pd.read_csv(fp)

df.head()

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,Jan 01 2015,22534.95,23063.83,22296.16,22720.48,15569776.0,7.877863e+11
1,Jan 02 2015,57047.79,57254.07,56526.45,57015.36,41942747.0,4.085053e+11
2,Jan 03 2015,43946.44,44086.61,43093.73,43502.15,26950379.0,9.393686e+11
3,Jan 04 2015,35959.64,36686.08,35560.69,36378.15,43135834.0,9.657302e+11
4,Jan 05 2015,9445.52,NaN,9153.66,9925.85,40960650.0,1.349320e+12


Nous allons illustrer le concept de `incept` à travers le renommage des noms d'entêtes de colonnes. Mais avant, nous allons créer deux fonctions que nous utiliserons dans nos exemples. Une première fonction ***`sansEspace()`*** pour remplacer l'espace dans es noms par un ***`_`*** un underscore. La seconde fonction ***`sansAsterix()`*** quant à elle permettra de supprimer les astérix dans les noms.

Remplace dans une chaîne de caractère, un espace `" "` par un underscore `"_"`

In [25]:
def sansEspace(col_name: str) -> str:
    return col_name.replace(' ','_')

Remplace dans une chaîne de caractère, un astérix `*` par rien du tout.

In [ ]:
def sansAsterix(col_name: str) -> str:
    return col_name.replace('*','')

## 1 - Comportement par défaut (sans `inplace` ou `inplace = False`)

Par défaut, les méthodes pandas **retournent un nouvel objet**.
Le DataFrame original n'est pas modifié tant qu'on ne réassigne pas le résultat. En arrière-plan, cela touche au concept de mutabilité des objets en Python: un DataFrame est un objet mutable.

In [28]:
df.rename(sansEspace, axis = 'columns')
df.rename(sansAsterix, axis= 'columns')
df.columns

Index(['Date', 'Open*', 'High', 'Low', 'Close**', 'Volume', 'Market Cap'], dtype='object')

On constate que les entêtes des colonnes ont toujours les ***`espaces`*** et les ***`astérix`***, le dataframe n'a pas été modifié.

## 2 -Utilisation avec assignation (recommandé)
Afin de rendre permanent cette modification, il va falloir stocker le dataframe dans une variable

In [29]:
df = df.rename(sansEspace, axis = 'columns') #  dont le nom contient un espace
df = df.rename(sansAsterix, axis= 'columns') # dont le nom contient un astérix
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market_Cap'], dtype='object')

Ici, nous réassignons explicitement le résultat. Le DataFrame est maintenant modifié de manière claire et prévisible. On constate que les ***`espaces`*** et ***`astérix`*** ont disparu des entêtes des colonnes.

## 3 - Utilisation avec `inplace=True`

In [35]:
df.rename(str.upper, axis = 'columns', inplace = True)
df.columns

Index(['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'MARKET_CAP'], dtype='object')

Avec `inplace=True`, le DataFrame est modifié directement, sans avoir besoin d’une réassignation. Les noms des entêtes ont été renommés en majuscule.

## 4️ - Le piège classique : retour de `None`

In [41]:
toy = pd.DataFrame({"A": [1, 2, 3], "B": ['i', 'j', 'k']})
result = toy.drop(0, inplace=True)
result

Les méthodes utilisant `inplace=True` retournent `None`.

Il est donc incorrect d’écrire :

```python
toy = toy.drop(0, inplace=True)
```
Cela écraserait l'objet avec `None`.

#### En résumé, on a que deux cas d'usage lorsqu'on veut modifier un datafeame
- On veut modifier le dataframe, observer le résultat de la modification, en conservant une copie de l'original
- On veut mofifier le dtaframe pour de bon sans garder une copie

#### Ceci nous conduit 4 cas d'utilisation:
    - Comment expérimenter une modification avec inplace= True
    - Comment expérimenter une modification avec inplace = False
    - Comment effectuer une modification pour de bon et la conserver avec inplace =True
    - Comment effectuer une modification pour de bon et la conserver avec inplace = False

## 5 - Faut-il utiliser `inplace` ?

Bien que `inplace=True` semble plus simple,
il casse le chaînage d'opérations et peut rendre le code moins prévisible.

De plus, dans les versions récentes de pandas,
le gain mémoire n’est pas toujours significatif.

En pratique, il est souvent recommandé de privilégier :

```python
df = df.dropna()
```

plutôt que :

```python
df.dropna(inplace=True)
```

Cela favorise également la lisibilité et la reproductibilité du code.

## 6 - À retenir

- `inplace=True` modifie l'objet original.
- Les méthodes avec `inplace` retournent `None`.
- Attention aux affectations accidentelles.
- L’écriture explicite (`df = df.method()`) est souvent préférable.